In [3]:
# imports
import pandas as pd
import geopandas as gpd
import requests
import numpy as np

In [44]:
def load_census_tract():
    """
    Data description link: https://docs.digital.mass.gov/dataset/massgis-data-datalayers-2010-us-census
    """
    file = gpd.read_file("../shapefiles/tl_2019_25_tract/tl_2019_25_tract.shp")
    return file

def get_median_hh_income():
    '''
        Returns Pandas DataFrame representation Median Household Income Estimate by Census Tract for MA.
        American Community Survey (ACS) 2018 Census data used.
        Specific table: ACS 2018 5-year detailed table "B19013_001E"
    '''
    URL = "https://api.census.gov/data/2018/acs/acs5?get=B19013_001E&for=tract:*&in=state:25"

    response = requests.get(url = URL)
    data = response.json()
    
    median_income_df = pd.DataFrame(data[1:len(data)-1], columns = data[0])
    
    return median_income_df

def prepend_zeros(string):
    # prepends zeros until string length is at least 6
    string = string.replace(' ', '')
    while len(string) < 6:
        string = '0' + string
    return string
    

In [59]:
tract_df = load_census_tract()
income_df = pd.read_csv('./output/stops_with_incomeLevel.csv').drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
income_df['TRACT_ID'] = income_df['TRACT_ID'].astype(str)

income_df['TRACT_ID'] = income_df['TRACT_ID'].apply(lambda x: prepend_zeros(x))
tract_df = tract_df[['GEOID', 'geometry']]
tract_df['GEOID'] = tract_df['GEOID'].apply(lambda x: x[-6:])

In [63]:
pd.merge(income_df, tract_df, left_on='TRACT_ID', right_on='GEOID').tocsv()

,STOP_ID,STOP_NAME,TOWN,TOWN_ID,TRACT_ID,income,income_level,GEOID,geometry
0,3077,Gallivan Blvd @ opp Marsh St,BOSTON,35,100700,87063,2,100700,"POLYGON ((-71.05890 42.28200, -71.05860 42.282..."
1,33076,Gallivan Blvd @ Hallet St,BOSTON,35,100700,87063,2,100700,"POLYGON ((-71.05890 42.28200, -71.05860 42.282..."
2,8814,Hilltop St @ Hallet St,BOSTON,35,100700,87063,2,100700,"POLYGON ((-71.05890 42.28200, -71.05860 42.282..."
3,48812,Hallet St @ Hilltop St,BOSTON,35,100700,87063,2,100700,"POLYGON ((-71.05890 42.28200, -71.05860 42.282..."
4,3371,Gallivan Blvd @ Frederika St,BOSTON,35,100700,87063,2,100700,"POLYGON ((-71.05890 42.28200, -71.05860 42.282..."
...,...,...,...,...,...,...,...,...,...
7866,2742,Broadway @ Mason St,SOMERVILLE,274,350600,152689,3,350600,"POLYGON ((-71.12725 42.40329, -71.12673 42.404..."
7867,2381,College Ave @ Powder House Sq,SOMERVILLE,274,350600,152689,3,350600,"POLYGON ((-71.12725 42.40329, -71.12673 42.404..."
7868,2405,College Ave @ Warner St,SOMERVILLE,274,350600,152689,3,350600,"POLYGON ((-71.12725 42.40329, -71.12673 42.404..."
7869,2691,Broadway opp Packard Ave,SOMERVILLE,274,350600,152689,3,350600,"POLYGON ((-71.12725 42.40329, -71.12673 42.404..."


Unnamed: 0                           571
Unnamed: 0.1                         571
STOP_ID                             1512
STOP_NAME       Bowdoin St @ Hamilton St
TOWN                              BOSTON
TOWN_ID                               35
TRACT_ID                          091700
income                             43125
income_level                           2
Name: 571, dtype: object

0       100700
1       130200
2       100300
3       130200
4       100800
         ...  
7805    336300
7806    336300
7807    336402
7808    336402
7809    336300
Name: TRACT_ID, Length: 7810, dtype: int64